# 演習8

In [1]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000

1000

In [2]:
using DataFrames, DataFramesMeta, CSV, Dates, StatsBase, CategoricalArrays, Distributions

In [3]:
# 演習2
return_data_df = CSV.read("../../data/ch08_return_data.csv", DataFrame, missingstring=["NA"])
market_return_data_df = CSV.read("../../data/ch08_market_return_data.csv", DataFrame, missingstring=["NA"])
event_data_df = CSV.read("../../data/ch08_event_data.csv", DataFrame, missingstring=["NA"])

## 予想サプライズの計算とグルーピング
event_data_df = @chain event_data_df begin
    @transform(:event_ID = 1:size(event_data_df)[1])
    @rtransform begin
        :surp = (:earnings_forecast - :realized_earnings) / :lagged_ME
        :year = Dates.Year(:event_date).value
    end
    groupby(:year)
    @transform(:surp_rank = cut(:surp, 5, labels=1:5))
end

## 日付インデックスを作成
N_DAYS = market_return_data_df.date |> unique |> length
N_FIRMS = return_data_df.firm_ID |> unique |> length
N_EVENTS = event_data_df.event_date |> unique |> length

date_ID_df = DataFrame(
    :date => market_return_data_df.date |> unique |> sort,
    :date_ID => 1:N_DAYS,
)

market_return_data_df = innerjoin(market_return_data_df, date_ID_df, on = :date)

return_data_df = @chain return_data_df begin
    innerjoin(date_ID_df, on = [:date])
    @select($(Not(:date)))
    innerjoin(market_return_data_df, on = [:date_ID])
end

event_data_df = @chain event_data_df begin
    innerjoin(date_ID_df, on = [:event_date => :date])
    @select(:firm_ID, :surp, :surp_rank, :date_ID, :event_date, :year, :event_ID)
end

full_sample_data_df = DataFrame(
    :event_ID => repeat(1:N_EVENTS, (100 + 30 + 1 + 30)) |> sort,
    :relative_days => repeat(-(100 + 30):30, N_EVENTS)
)

@chain event_data_df begin
    innerjoin(full_sample_data_df, on = :event_ID)
    rename(:date_ID => :base_date_ID)
    @rtransform begin
        :date_ID = :base_date_ID + :relative_days + 1
    end
    innerjoin(return_data_df, on = [:date_ID])
end

,firm_ID,event_date,realized_earnings,earnings_forecast,lagged_ME
,Int64,Date,Float64,Float64,Float64
1,1,2017-05-08,421.79,500.38,8285.87
2,2,2017-08-28,131.49,144.13,1551.31
3,3,2017-10-17,459.3,463.03,3154.17
4,4,2017-10-17,976.94,1298.48,19323.2
5,5,2017-02-09,706.28,779.63,9351.8
6,6,2017-05-23,530.62,544.65,7558.49
7,7,2017-05-16,613.26,717.0,11947.4
8,8,2017-03-08,142.95,157.86,2215.57
9,9,2017-10-10,802.61,792.09,13913.7
